## 14. 분산형 공유 변수
- 저수준 API는 RDD 뿐만 아니라 분산형 공유 변수도 있음
- 분산형 공유 변수에는 브로드캐스트 변수와 어큐뮬레이터라는 타입이 있음
- 어큐뮬레이터를 사용하면 모든 태스크의 데이터를 공유 결과에 추가할 수 있음 (ex: 잡의 입력 레코드를 파싱하면서 얼마나 많은 오류가 발생했는지 확인하는 카운터를 구현할 수 있음
- 브로드캐스트 변수를 사용하면 모든 워커 노드에 큰 값을 저장하므로 재전송 없이 많은 스파크 액션에서 재사용할 수 있음

### 14.1 브로드캐스트 변수
- 브로드캐스트 변수는 변하지 않는 값(불변성 값)을 클로저 함수의 변수로 캡슐화하지 않고 클러스터에서 효율적으로 공유하는 방법을 제공함
- 태스크에서 드라이버 노드의 변수를 사용할 때 클로저 함수 내부에서 단순하게 참조하는 방법을 사용 -> 룩업 테이블이나 머신러닝 모델 같은 큰 변수를 사용하는 경우 비효율적임.
 - 이유는 클로저 함수에서 변수를 사용할 때 워커 노드에서 여러 번(태스크 당 한번) 역직렬화가 일어나기 때문임
 - 여러 스파크 액션과 잡에서 동일한 변수를 사용하면 잡을 실행할 때마다 워커로 큰 변수를 재전송함
 - 이 문제를 해결하기 위해 브로드캐스트 변수 사용
- 브로드캐스트 변수는 모든 태스크마다 직렬화하지 않고 클러스터의 모든 머신에 캐시하는 불변성 공유 변수


In [4]:
my_collection = 'Spark The Definitive Guide : Big Data Processing Made Simple'.split(' ')
words = spark.sparkContext.parallelize(my_collection, 2)
words.collect()

['Spark',
 'The',
 'Definitive',
 'Guide',
 ':',
 'Big',
 'Data',
 'Processing',
 'Made',
 'Simple']

In [5]:
# 수 킬로바이트, 메가바이트, 기가바이트 크기를 가진 다른 정보와 함께 단어 목록을 추가하는 과정
# 이 구조체를 스파크에 브로드캐스트할 수 있음
supplementalData = {'Spark':1000, 'Definitive':200, 'Big':-300, 'Simple':100}

In [10]:
# value 매서드를 통해 직렬화된 함수에서 브로드캐스트된 데이터를 직렬화 하지 않아도 접근할 수 있음
# 스파크는 브로드캐스트 기능을 통해 데이터를 보다 효율적으로 전송하므로 직렬화와 역직렬화에 대한 부하를 크게 줄일 수 있음
suppBroadcast = spark.sparkContext.broadcast(supplementalData)
suppBroadcast.value

{'Spark': 1000, 'Definitive': 200, 'Big': -300, 'Simple': 100}

In [11]:
# 브로드캐스트된 데이터를 RDD로 변환하는 과정
words.map(lambda word: (word, suppBroadcast.value.get(word,0))).sortBy(lambda wordPair: wordPair[1]).collect()

[('Big', -300),
 ('The', 0),
 ('Guide', 0),
 (':', 0),
 ('Data', 0),
 ('Processing', 0),
 ('Made', 0),
 ('Simple', 100),
 ('Definitive', 200),
 ('Spark', 1000)]

- 브로드캐스트 변수를 사용한 방식과 클로저에 담아 전달하는 방식의 유일한 차이점은 브로드캐스트 변수를 사용하는 것이 더 효율적이기 때문임.
- 브로드캐스트 변수에 작은 크기의 딕셔너리 타입을 사용한다면 큰 부하가 발생하지 않음
- 다만 큰 크기의 데이터를 사용할 경우라면 전체 테스크에서 데이터를 직렬화하는 데 발생하는 부하가 매우 커질 수 있음
- RDD, UDF, Dataset에서도 사용가능하고 동일한 성능을 얻을 수 있음

### 14.2 어큐뮬레이터
- 어큐뮬레이터는 트랜스포메이션 내부의 다양한 값을 갱신하는데 사용함 
- 내고장성을 보장하면서 효율적인 방식으로 드라이버에 값을 전달할 수 있음
- 어큐뮬레이터는 스파크 클러스터에서 로우 단위로 안전하게 값을 갱신할 수 있는 변경 가능한 변수를 제공
- 디버깅용이나 저수준 집게 생성용으로 사용할 수 있음
- 어큐뮬레이터는 결합성과 가환성을 가진 연산을 통해서만 더할 수 있는 변수이므로 병렬 처리 과정에서 효율적으로 사용할 수 있음 
- 카운터(맵리듀스의 카운터와 같은)나 합계를 구하는 용도로 사용할 수 있음
- 어큐뮬레이터의 값은 액션을 처리하는 과정에서만 갱신됨
- 스파크는 각 테스크에서 어큐뮬레이터를 한 번만 갱신하도록 제어함 
- 어큐뮬레이터는 스파크의 지연 연산 모델에 영향을 주지 않음 
- 어큐뮬레이터가 RDD 처리중에 갱신되면 RDD 연산이 실제로 수행된 시점, 즉 특정 RDD나 그 RDD의 부모 RDD에 액션을 실행하는 시점에 딱 한 번만 값을 갱신함
- map 함수 같은 지연 처리 형태의 트랜스포메이션에서 어큐뮬레이터 갱신 작업을 수행하는 경우 실제 실행 전까지는 어큐뮬레이터가 갱신되지 않음


In [43]:
# Dataset API
flights = spark.read.parquet('file:///home/ubuntu/Spark-The-Definitive-Guide/data/flight-data/parquet/2010-summary.parquet')
flights

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

In [44]:
# SQL로도 충분히 처리할 수 있긴 함
# sparkContext로 가장 간단한 어큐뮬레이터 생성 가능
accChina = spark.sparkContext.accumulator(0)
accChina

Accumulator<id=6, value=0>

- 함수의 파라미터로 문자열값을 전달하거나 register 함수의 두 번째 파라미터를 사용해 이름을 지정할 수 있음
- 실행 결과는 스파크 UI에서 확인 가능, 이름이 지정되지 않으면 확인불가


In [45]:
# 어큐뮬레이터로 값을 더하는 방법
def accChinaFunc(flight_row):
  destination = flight_row["DEST_COUNTRY_NAME"]
  origin = flight_row["ORIGIN_COUNTRY_NAME"]
  if destination == "China":
    accChina.add(flight_row["count"])
  if origin == "China":
    accChina.add(flight_row["count"])

In [46]:
# foreach 매서드는 액션이고, 스파크는 액션에서만 어큐뮬레이터의 실행을 보장하기 때문에 이 메서드 사용
flights.foreach(lambda flight_row: accChinaFunc(flight_row))

In [47]:
accChina.value

953

### 14.2.2 사용자 정의 어큐뮬레이터
- 파이썬 사용자는 AccumulatorParam을 상속받아서 진행할 수 있음